In [4]:
!pip install flask_sqlalchemy


     -------------------------------------- 96.9/96.9 kB 503.6 kB/s eta 0:00:00
     -------------------------------------- 96.6/96.6 kB 461.8 kB/s eta 0:00:00
     ------------------------------------ 133.1/133.1 kB 437.2 kB/s eta 0:00:00
     ------------------------------------ 242.5/242.5 kB 479.8 kB/s eta 0:00:00
  Attempting uninstall: MarkupSafe
    Found existing installation: MarkupSafe 2.0.1
    Uninstalling MarkupSafe-2.0.1:
      Successfully uninstalled MarkupSafe-2.0.1
  Attempting uninstall: itsdangerous
    Found existing installation: itsdangerous 2.0.1
    Uninstalling itsdangerous-2.0.1:
      Successfully uninstalled itsdangerous-2.0.1
  Attempting uninstall: click
    Found existing installation: click 8.0.4
    Uninstalling click-8.0.4:
      Successfully uninstalled click-8.0.4
  Attempting uninstall: Werkzeug
    Found existing installation: Werkzeug 2.0.3
    Uninstalling Werkzeug-2.0.3:
      Successfully uninstalled Werkzeug-2.0.3
  Attempting uninstall: Jinj

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
anaconda-project 0.11.1 requires ruamel-yaml, which is not installed.


In [5]:
from flask_sqlalchemy import SQLAlchemy

db = SQLAlchemy()

class Book(db.Model):
    id = db.Column(db.Integer, primary_key=True)
    title = db.Column(db.String(255), nullable=False)
    author = db.Column(db.String(255), nullable=False)
    stock = db.Column(db.Integer, nullable=False)

class Member(db.Model):
    id = db.Column(db.Integer, primary_key=True)
    name = db.Column(db.String(255), nullable=False)
    outstanding_debt = db.Column(db.Float, nullable=False, default=0.0)

class Transaction(db.Model):
    id = db.Column(db.Integer, primary_key=True)
    book_id = db.Column(db.Integer, db.ForeignKey('book.id'), nullable=False)
    member_id = db.Column(db.Integer, db.ForeignKey('member.id'), nullable=False)
    issue_date = db.Column(db.Date, nullable=False)
    return_date = db.Column(db.Date)
    rent_fee = db.Column(db.Float, nullable=False, default=0.0)


In [6]:
from flask import Flask, request, jsonify

app = Flask(__name__)

# Books routes
@app.route('/books', methods=['GET'])
def get_books():
    books = Book.query.all()
    return jsonify([book.serialize() for book in books])

@app.route('/books', methods=['POST'])
def create_book():
    data = request.json
    book = Book(title=data['title'], author=data['author'], stock=data['stock'])
    db.session.add(book)
    db.session.commit()
    return jsonify(book.serialize()), 201

# Members routes
@app.route('/members', methods=['GET'])
def get_members():
    members = Member.query.all()
    return jsonify([member.serialize() for member in members])

@app.route('/members', methods=['POST'])
def create_member():
    data = request.json
    member = Member(name=data['name'])
    db.session.add(member)
    db.session.commit()
    return jsonify(member.serialize()), 201


In [7]:
@app.route('/transactions', methods=['POST'])
def issue_book():
    data = request.json
    book_id = data['book_id']
    member_id = data['member_id']

    # Check if the book is available
   
